In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import guidance

In [3]:


# set the default language model used to execute guidance programs
guidance.llm = guidance.llms.OpenAI("text-davinci-003", temperature=1)



template = """Tweak this proverb to apply to model instructions instead.

{{proverb}}
- {{book}} {{chapter}}:{{verse}}

UPDATED
Where there is no guidance{{gen 'rewrite' stop="\\n-"}}
- GPT {{#select 'chapter'}}9{{or}}10{{or}}11{{/select}}:{{gen 'verse'}}"""
# define a guidance program that adapts a proverb
program = guidance( template )


# execute the program on a specific proverb
executed_program = program(
    proverb="Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.",
    book="Proverbs",
    chapter=1,
    verse=14
)

Stop program Tweak this proverb to apply to model instructions instead.

 Where there is no guidance, a people falls,
but in an abundance of counselors there is safety. 
- Proverbs 1 : 14 

UPDATED
Where there is no guidance , a model fails,
but in an abundance of instructions there is safety. 
- GPT 910119 : 1

In [4]:
executed_program["rewrite"]

', a model fails,\nbut in an abundance of instructions there is safety.'

In [5]:
# connect to a chat model like GPT-4 or Vicuna
gpt35 = guidance.llms.OpenAI("gpt-3.5-turbo")
# vicuna = guidance.llms.transformers.Vicuna("your_path/vicuna_13B", device_map="auto")

experts = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
{{~/user}}

{{#assistant~}}
{{gen 'expert_names' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}
''', llm=gpt35)

experts(query='How can I be more productive?')

Stop program system You are a helpful and terse assistant. user I want a response to the following question:
 How can I be more productive? 
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet. assistant Sure, here are three world-class experts who would be great at answering the question on how to be more productive:

1. David Allen - Known for his book "Getting Things Done," Allen is a productivity consultant and expert who has helped millions of people improve their productivity and organization skills.

2. Tim Ferriss - A renowned author and entrepreneur, Ferriss has written books like "The 4-Hour Workweek" and is known for his insights on productivity, time management, and efficiency.

3. Cal Newport - An author and professor, Newport focuses on the intersection of technology, productivity, and work-life balance. His book "Deep Work" provides valuable strategies for improving focus and productivity in a distracted world.

Now, let's move on to answering your question on how to be more productive. user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer. assistant To be more productive, consider the following advice from a collaboration of world-class experts:

1. Prioritize and organize: Start by clarifying your goals and breaking them down into actionable tasks. Use techniques like David Allen's "Getting Things Done" methodology to create a system for capturing and organizing your tasks and commitments.

2. Optimize your time: Tim Ferriss suggests focusing on the most important tasks that yield the highest results. Identify your most productive hours and schedule your most challenging work during that time. Use techniques like time blocking and batching similar tasks to maximize efficiency.

3. Embrace deep work: Cal Newport emphasizes the importance of eliminating distractions and creating a focused work environment. Practice deep work, which involves dedicating uninterrupted time to tasks that require intense concentration. Minimize interruptions from emails, social media, and other distractions to enhance productivity.

Remember, productivity is a personal journey, and what works for one person may not work for another. Experiment with different techniques and find a productivity system that suits your needs and preferences.

In [6]:

tdv = guidance.llms.OpenAI("text-davinci-003")
# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
character_maker = guidance("""The following is a character profile for an RPG game in JSON format.
```json
{
    "id": "{{id}}",
    "description": "{{description}}",
    "name": "{{gen 'name'}}",
    "age": {{gen 'age' pattern='[0-9]+' stop=','}},
    "armor": "{{#select 'armor'}}leather{{or}}chainmail{{or}}plate{{/select}}",
    "weapon": "{{select 'weapon' options=valid_weapons}}",
    "class": "{{gen 'class'}}",
    "mantra": "{{gen 'mantra' temperature=0.7}}",
    "strength": {{gen 'strength' pattern='[0-9]+' stop=','}},
    "items": [{{#geneach 'items' num_iterations=5 join=', '}}"{{gen 'this' temperature=0.7}}"{{/geneach}}]
}```""")

# generate a character
cm = character_maker(
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons, llm=tdv
)

Stop program The following is a character profile for an RPG game in JSON format.
```json
{
 "id": " {{id}} ",
 "description": " {{description}} ",
 "name": " {{gen 'name'}} ",
 "age": {{gen 'age' pattern='[0-9]+' stop=','}} ,
 "armor": " {{#select 'armor'}} leather {{or}} chainmail {{or}} plate {{/select}} ",
 "weapon": " {{select 'weapon' options=valid_weapons}} ",
 "class": " {{gen 'class'}} ",
 "mantra": " {{gen 'mantra' temperature=0.7}} ",
 "strength": {{gen 'strength' pattern='[0-9]+' stop=','}} ,
 "items": [ {{#geneach 'items' num_iterations=5 join=', '}} " {{gen 'this' temperature=0.7}} " {{/geneach}} ]
}```

In [17]:
cm.variables()

{'llm': <guidance.llms._openai.OpenAI at 0x10787a410>,
 'logging': False,
 'id': 'e1f491f7-7ab8-4dac-8c20-c92b5e7d883d',
 'description': 'A quick and nimble fighter.',
 'valid_weapons': ['sword', 'axe', 'mace', 'spear', 'bow', 'crossbow'],
 '@raw_prefix': 'The following is a character profile for an RPG game in JSON format.\n```json\n{\n    "id": "{{!--GMARKER_START_variable_ref$&#123;&#123;id&#125;&#125;$--}}e1f491f7-7ab8-4dac-8c20-c92b5e7d883d{{!--GMARKER_END_variable_ref$$--}}",\n    "description": "{{!--GMARKER_START_variable_ref$&#123;&#123;description&#125;&#125;$--}}A quick and nimble fighter.{{!--GMARKER_END_variable_ref$$--}}",\n    "name": "{{!--GMARKER_START_gen$&#123;&#123;gen \'name\'&#125;&#125;$--}}Kara",\n    "class": "Rogue",\n    "level": 10,\n    "stats": {\n        "strength": 8,\n        "dexterity": 16,\n        "constitution": 12,\n        "intelligence": 10,\n        "wisdom": 10,\n        "charisma": 14\n    },\n    "inventory": [\n        "dagger",\n        "s

In [8]:
result